# PeerDAS simulation

## Imports

In [ ]:
import subprocess
import time
import os

if not os.path.exists("raw_traces"):
    os.mkdir("raw_traces")

## Experiment 1: 
Setup
- 21 blobs (max)
- extended blob size: 256kB
- 128 cells per blob
-> col size: 42kB
- 1 publisher publishing 128 columns (producer)
- 20 nodes subscribing to 8 random columns (validators)

## Setup topics and participants

In [ ]:
num_validators = 21
num_topics = 8

In [ ]:
topics = [f'/eth2/fulu_fork_digest/simplified_data_column_sidecar_{i}/ssz_snappy' for i in range(num_topics)]

## Subscriptions

In [ ]:
subscribe_processes = []  # Keep track of all started processes
trace_files = []

for i,topic in enumerate(topics):
    trace_file = f"raw_traces/trace-data-col-{i}.tsv"
    trace_files.append(trace_file)
    proc = subprocess.Popen([
        "./p2p-multi-subscribe",
        "-topic", topic,
        "-ipfile", "ips.txt",
        "-output-trace", trace_file,
    ])
    subscribe_processes.append(proc)

print(f"Started {len(subscribe_processes)} subscriber processes")



In [ ]:
def publish_data(topic):
    return subprocess.Popen([
        "./p2p-multi-publish",
        "-topic", topic,
        "-ipfile", "ips.txt",
        "-start-index", "0",
        "-end-index", "1",
        "-datasize", "43008",
    ])

publish_procs = []
for topic in topics:
    proc = publish_data(topic)
    publish_procs.append(proc)

time.sleep(5)


## Merge raw traces

In [ ]:
merged_traces_file = "merged_traces.tsv"
with open(merged_traces_file, "w") as f:
    for trace_file in trace_files:
        if os.path.exists(trace_file):
            with open(trace_file, "r") as t:
                f.write(t.read())
        else:
            print(f"Warning: trace file {trace_file} not found. Skipping.")

time.sleep(5)

2025/12/03 00:20:27 recv error: rpc error: code = Unavailable desc = error reading from server: read tcp 10.192.58.90:60700->34.105.218.12:33212: read: connection reset by peer
2025/12/03 00:20:27 Stream closed. Total messages received: 0
2025/12/03 00:20:27 recv error: rpc error: code = Unavailable desc = error reading from server: read tcp 10.192.58.90:60695->34.40.18.225:33212: read: connection reset by peer
2025/12/03 00:20:27 Stream closed. Total messages received: 0
2025/12/03 00:20:27 recv error: rpc error: code = Unavailable desc = error reading from server: read tcp 10.192.58.90:60688->34.65.254.125:33212: read: connection reset by peer
2025/12/03 00:20:27 Stream closed. Total messages received: 0
2025/12/03 00:20:28 recv error: rpc error: code = Unavailable desc = error reading from server: read tcp 10.192.58.90:60701->34.65.64.154:33212: read: connection reset by peer
2025/12/03 00:20:28 Stream closed. Total messages received: 0
2025/12/03 00:20:28 recv error: rpc error: cod

In [ ]:
for proc in subscribe_processes:
    proc.terminate()